In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
from scipy.stats import scoreatpercentile as pct
from scipy.stats import iqr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE  ## recursive feature elemination technique
pd.options.display.max_columns = 200

In [90]:
ica_df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv", index_col = 0)
ica_df = ica_df.drop('effective_to_date', axis=1)
ica_df

,region,customer_lifetime_value,response,coverage,education,month,employment_status,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,central,4809,no,basic,college,feb,employed,m,48029,suburban,married,61,7,52,0,9,corporate auto,corporate l3,offer3,agent,292,four-door car,medsize
1,west region,2228,no,basic,college,jan,unemployed,f,26794,suburban,single,64,3,26,0,1,personal auto,personal l3,offer4,call center,744,four-door car,medsize
2,east,14947,no,basic,bachelor,feb,employed,m,22139,suburban,single,100,34,31,0,2,personal auto,personal l3,offer3,call center,480,suv,medsize
3,north west,22332,yes,extended,college,jan,employed,m,49078,suburban,single,97,10,3,0,2,corporate auto,corporate l3,offer2,branch,484,four-door car,medsize
4,north west,9025,no,premium,bachelor,jan,medical leave,f,23675,suburban,married,117,15,31,1,7,personal auto,personal l2,offer1,branch,707,four-door car,medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,central,15563,no,premium,bachelor,jan,unemployed,f,64811,suburban,married,253,8,40,2,7,personal auto,personal l1,offer3,web,1214,luxury car,medsize
10685,north west,5259,no,basic,college,jan,employed,f,61146,urban,married,65,7,68,0,6,personal auto,personal l3,offer2,branch,273,four-door car,medsize
10686,central,23893,no,extended,bachelor,feb,employed,f,39837,rural,married,201,11,63,0,2,corporate auto,corporate l3,offer1,web,381,luxury suv,medsize
10687,west region,11971,no,premium,college,feb,employed,f,64195,urban,divorced,158,0,27,4,6,personal auto,personal l1,offer1,branch,618,suv,medsize


In [91]:
categoricals_df = ica_df.select_dtypes('object')
categoricals_df

,region,response,coverage,education,month,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,central,no,basic,college,feb,employed,m,suburban,married,corporate auto,corporate l3,offer3,agent,four-door car,medsize
1,west region,no,basic,college,jan,unemployed,f,suburban,single,personal auto,personal l3,offer4,call center,four-door car,medsize
2,east,no,basic,bachelor,feb,employed,m,suburban,single,personal auto,personal l3,offer3,call center,suv,medsize
3,north west,yes,extended,college,jan,employed,m,suburban,single,corporate auto,corporate l3,offer2,branch,four-door car,medsize
4,north west,no,premium,bachelor,jan,medical leave,f,suburban,married,personal auto,personal l2,offer1,branch,four-door car,medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,central,no,premium,bachelor,jan,unemployed,f,suburban,married,personal auto,personal l1,offer3,web,luxury car,medsize
10685,north west,no,basic,college,jan,employed,f,urban,married,personal auto,personal l3,offer2,branch,four-door car,medsize
10686,central,no,extended,bachelor,feb,employed,f,rural,married,corporate auto,corporate l3,offer1,web,luxury suv,medsize
10687,west region,no,premium,college,feb,employed,f,urban,divorced,personal auto,personal l1,offer1,branch,suv,medsize


In [92]:
categoricals_df.education.unique()


array(['college', 'bachelor', 'high school or below', 'doctor', 'master'],
      dtype=object)

In [93]:
categoricals_df.policy.unique()

array(['corporate l3', 'personal l3', 'personal l2', 'corporate l2',
       'personal l1', 'special l1', 'corporate l1', 'special l3',
       'special l2'], dtype=object)

In [94]:
categoricals_df['policy'] = categoricals_df['policy'].replace({'corporate l3':'corporate', 
                                                               'corporate l2':'corporate',
                                                               'corporate l1': 'corporate',
                                                              "personal l3": "personal",
                                                              "personal l2": "personal",
                                                              "personal l1": "personal",
                                                              'special l1': 'special',
                                                              'special l2': 'special',
                                                              'special l3': 'special'})
categoricals_df.policy.unique()

array(['corporate', 'personal', 'special'], dtype=object)

In [95]:
categoricals_df.vehicle_class.value_counts()


four-door car    5416
two-door car     2193
suv              2109
sports car        564
luxury suv        214
luxury car        193
Name: vehicle_class, dtype: int64

In [96]:
categoricals_df['vehicle_class'] = categoricals_df['vehicle_class'].replace({'sports car':'luxury car', 
                                                               'luxury suv':'luxury car',})
categoricals_df.vehicle_class.value_counts()

four-door car    5416
two-door car     2193
suv              2109
luxury car        971
Name: vehicle_class, dtype: int64

#### One Hot encoding of categorical variables
#### categoricals=pd.get_dummies(categoricals, drop_first=True)

categoricals_df= pd.get_dummies(ica_df[["region","response", "coverage","education","effective_to_date","month", "employment_status",
                       "gender","location_code","marital_status", "policy_type", "policy","renew_offer_type", "sales_channel"
                      , "vehicle_class", "vehicle_size"]], drop_first=True)

In [97]:
categoricals_df= pd.get_dummies(categoricals_df, drop_first=True)
categoricals_df

,region_east,region_north west,region_west region,response_yes,coverage_extended,coverage_premium,education_college,education_doctor,education_high school or below,education_master,month_jan,employment_status_employed,employment_status_medical leave,employment_status_retired,employment_status_unemployed,gender_m,location_code_suburban,location_code_urban,marital_status_married,marital_status_single,policy_type_personal auto,policy_type_special auto,policy_personal,policy_special,renew_offer_type_offer2,renew_offer_type_offer3,renew_offer_type_offer4,sales_channel_branch,sales_channel_call center,sales_channel_web,vehicle_class_luxury car,vehicle_class_suv,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small
0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0
3,0,1,0,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
4,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,1,1,0,0,1,0
10685,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0
10686,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0
10687,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0


In [98]:
numerical_df = ica_df._get_numeric_data()
numerical_df

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,4809,48029,61,7,52,0,9,292
1,2228,26794,64,3,26,0,1,744
2,14947,22139,100,34,31,0,2,480
3,22332,49078,97,10,3,0,2,484
4,9025,23675,117,15,31,1,7,707
...,...,...,...,...,...,...,...,...
10684,15563,64811,253,8,40,2,7,1214
10685,5259,61146,65,7,68,0,6,273
10686,23893,39837,201,11,63,0,2,381
10687,11971,64195,158,0,27,4,6,618


In [131]:
# X Y split
X = numerical_df.drop('total_claim_amount', axis=1)
y = numerical_df.total_claim_amount


In [132]:
X

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,4809,48029,61,7,52,0,9
1,2228,26794,64,3,26,0,1
2,14947,22139,100,34,31,0,2
3,22332,49078,97,10,3,0,2
4,9025,23675,117,15,31,1,7
...,...,...,...,...,...,...,...
10684,15563,64811,253,8,40,2,7
10685,5259,61146,65,7,68,0,6
10686,23893,39837,201,11,63,0,2
10687,11971,64195,158,0,27,4,6


In [133]:
X = pd.concat([categoricals_df, X], axis = 1)


In [134]:
X

,region_east,region_north west,region_west region,response_yes,coverage_extended,coverage_premium,education_college,education_doctor,education_high school or below,education_master,month_jan,employment_status_employed,employment_status_medical leave,employment_status_retired,employment_status_unemployed,gender_m,location_code_suburban,location_code_urban,marital_status_married,marital_status_single,policy_type_personal auto,policy_type_special auto,policy_personal,policy_special,renew_offer_type_offer2,renew_offer_type_offer3,renew_offer_type_offer4,sales_channel_branch,sales_channel_call center,sales_channel_web,vehicle_class_luxury car,vehicle_class_suv,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,4809,48029,61,7,52,0,9
1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,2228,26794,64,3,26,0,1
2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,14947,22139,100,34,31,0,2
3,0,1,0,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,22332,49078,97,10,3,0,2
4,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,9025,23675,117,15,31,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,1,1,0,0,1,0,15563,64811,253,8,40,2,7
10685,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,5259,61146,65,7,68,0,6
10686,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,23893,39837,201,11,63,0,2
10687,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,11971,64195,158,0,27,4,6


X = pd.concat([X,y], axis = 1)

X.to_csv("new customer.csv", index = True)

In [135]:
# Train-test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30,random_state=123)

In [136]:
#Standardize the data.
std_scaler = StandardScaler().fit(X_train)   
X_train_scaled = std_scaler.transform(X_train)

In [137]:
X_test_scaled=std_scaler.transform(X_test)

In [138]:
X_train.shape

(7482, 42)

### Linear Regression Model

In [139]:
model=LinearRegression()
model.fit(X_train_scaled, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train_scaled, y_train)}, Test -> {model.score(X_test_scaled, y_test)}")

LinearRegression: Train -> 0.7702199933196456, Test -> 0.7702900776026751


### Lasso Model

In [140]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model=Lasso(alpha=0.05)

model.fit(X_train_scaled, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train_scaled, y_train)}, Test -> {model.score(X_test_scaled, y_test)}")

Lasso: Train -> 0.7702176858369268, Test -> 0.7703562125264327


### Ridge

In [141]:
model=Ridge(alpha=10000)
model.fit(X_train_scaled, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train_scaled, y_train)}, Test -> {model.score(X_test_scaled, y_test)}")

Ridge: Train -> 0.5695938395128315, Test -> 0.5720508437689427


### P-Value

In [142]:
X_added_constant = sm.add_constant(X)
X_added_constant

,const,region_east,region_north west,region_west region,response_yes,coverage_extended,coverage_premium,education_college,education_doctor,education_high school or below,education_master,month_jan,employment_status_employed,employment_status_medical leave,employment_status_retired,employment_status_unemployed,gender_m,location_code_suburban,location_code_urban,marital_status_married,marital_status_single,policy_type_personal auto,policy_type_special auto,policy_personal,policy_special,renew_offer_type_offer2,renew_offer_type_offer3,renew_offer_type_offer4,sales_channel_branch,sales_channel_call center,sales_channel_web,vehicle_class_luxury car,vehicle_class_suv,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,1.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,4809,48029,61,7,52,0,9
1,1.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,2228,26794,64,3,26,0,1
2,1.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,14947,22139,100,34,31,0,2
3,1.0,0,1,0,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,22332,49078,97,10,3,0,2
4,1.0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,9025,23675,117,15,31,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,1.0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,1,1,0,0,1,0,15563,64811,253,8,40,2,7
10685,1.0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,5259,61146,65,7,68,0,6
10686,1.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,23893,39837,201,11,63,0,2
10687,1.0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,11971,64195,158,0,27,4,6


In [144]:
model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[0:].index.tolist()
significant_features=X_added_constant[model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[0:].index.tolist()]
significant_features

,const,response_yes,coverage_extended,coverage_premium,education_college,education_high school or below,employment_status_unemployed,gender_m,location_code_suburban,location_code_urban,marital_status_single,renew_offer_type_offer2,renew_offer_type_offer3,vehicle_class_luxury car,vehicle_class_suv,income,monthly_premium_auto,months_since_last_claim
0,1.0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,48029,61,7
1,1.0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,26794,64,3
2,1.0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,22139,100,34
3,1.0,1,1,0,1,0,0,1,1,0,1,1,0,0,0,49078,97,10
4,1.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,23675,117,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,1.0,0,0,1,0,0,1,0,1,0,0,0,1,1,0,64811,253,8
10685,1.0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,61146,65,7
10686,1.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,39837,201,11
10687,1.0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,64195,158,0


In [156]:
# we need to add this constant value of 1 for the intercepts
model = sm.OLS(y,significant_features).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     2103.
Date:                Wed, 04 May 2022   Prob (F-statistic):               0.00
Time:                        16:19:12   Log-Likelihood:                -68008.
No. Observations:               10689   AIC:                         1.361e+05
Df Residuals:                   10671   BIC:                         1.362e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           -375.9946      7.700    -48.830      0.000    -391.088    -360.901
response_yes                     -24.8382      4.216     -5.892      0.000     -33.102     -16.575
coverage_extended                -16.8317      3.461     -4.863      0.000     -23.616     -10.048
coverage_premium                 -20.6085      6.272     -3.286      0.001     -32.902      -8.315
education_college                 -3.8640      3.275     -1.180      0.238     -10.284       2.555
education_high school or below    14.5548      3.332      4.369      0.000       8.024      21.085
employment_status_unemployed      87.4989      3.647     23.991      0.000      80.350      94.648
gender_m                          15.6427      2.732      5.727      0.000      10.288      20.997
location_code_suburban           381.9631      3.831     99.700      0.000     374.453     389.473
location_code_urban              222.3344      4.481     49.621      0.000     213.551     231.117
marital_status_single             70.2531      3.364     20.883      0.000      63.659      76.847
renew_offer_type_offer2           13.3536      3.131      4.265      0.000       7.216      19.491
renew_offer_type_offer3            6.4390      3.903      1.650      0.099      -1.211      14.089
vehicle_class_luxury car         -25.9544      7.774     -3.339      0.001     -41.192     -10.717
vehicle_class_suv                -23.0986      4.733     -4.881      0.000     -32.375     -13.822
income                            -0.0003   5.69e-05     -5.707      0.000      -0.000      -0.000
monthly_premium_auto               5.3779      0.079     67.708      0.000       5.222       5.534
months_since_last_claim            0.3706      0.135      2.747      0.006       0.106       0.635
==============================================================================
Omnibus:                     4761.287   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            59949.455
Skew:                           1.799   Prob(JB):                         0.00
Kurtosis:                      14.030   Cond. No.                     4.56e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.56e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Wrapper

In [148]:
X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)

#X_train.isna().sum()
nulls = pd.DataFrame(X_train.isna().sum()).reset_index()
#nulls.head()
nulls.columns = ['Column','nas']
#nulls.head()
#nulls[nulls['nas'] > 0].head()
cols_to_drop = nulls[nulls['nas'] > 0]['Column'] # Too drastic, but made on pourpose for quick filtering (don't do this in production!!)

X_train.drop(columns=cols_to_drop, axis=1, inplace = True)
X_test.drop(columns=cols_to_drop, axis=1, inplace = True)

#display(X_train)

lm = LinearRegression()

selector = RFE(lm, n_features_to_select= 8, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final selected features: ")
display(X_train)

Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 fe

,coverage_extended,coverage_premium,employment_status_unemployed,location_code_suburban,location_code_urban,marital_status_single,vehicle_class_luxury car,vehicle_class_suv
0,0,0,0,1,0,1,0,0
1,0,0,1,1,0,0,0,0
2,0,0,0,0,1,0,0,1
3,1,0,0,1,0,1,0,1
4,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...
7477,1,0,0,0,0,0,0,0
7478,0,0,0,1,0,0,0,0
7479,0,0,0,1,0,0,0,0
7480,0,0,0,0,0,0,0,0


### Embedded Method


In [154]:

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train=imp_mean.fit_transform(X_train)

In [155]:
X_train

array([[ 0.,  0.,  0., ..., 90.,  1.,  2.],
       [ 0.,  0.,  0., ..., 34.,  0.,  1.],
       [ 0.,  1.,  0., ..., 96.,  0.,  1.],
       ...,
       [ 0.,  0.,  1., ...,  8.,  4.,  8.],
       [ 0.,  1.,  0., ..., 90.,  0.,  2.],
       [ 0.,  0.,  0., ..., 54.,  0.,  7.]])